In [ ]:
0# Nhập thư viện pandas và tiếp tục xử lý file
import pandas as pd
file_path = "/content/train.crash"
# Đọc dữ liệu từ file
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()

# Tách từng phần dựa trên cấu trúc mô tả
import re

# Chuẩn bị danh sách để lưu trữ dữ liệu
data = []

# Xử lý từng phần, tách theo mẫu train_XXXXXX và nội dung theo sau
parts = re.split(r'train_\d+\n', content)[1:]  # Bỏ qua phần tử đầu tiên vì nó là rỗng
for part in parts:
    if part.strip():  # Kiểm tra phần không rỗng
        # Tách nội dung và nhãn
        review, label = part.rsplit(maxsplit=1)
        review = review.strip().strip('"')  # Xóa dấu nháy và khoảng trắng thừa
        label = label.strip()
        data.append([review, int(label)])

# Tạo DataFrame từ dữ liệu
df = pd.DataFrame(data, columns=['Review', 'Emotion'])

# Hiển thị vài dòng đầu của DataFrame để kiểm tra
df.head()


,Review,Emotion
0,Dung dc sp tot cam on \nshop Đóng gói sản phẩm...,0
1,Chất lượng sản phẩm tuyệt vời . Son mịn nhưng...,0
2,Chất lượng sản phẩm tuyệt vời nhưng k có hộp ...,0
3,:(( Mình hơi thất vọng 1 chút vì mình đã kỳ vọ...,1
4,Lần trước mình mua áo gió màu hồng rất ok mà đ...,1


In [ ]:
!pip install pycaret
!pip install pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 44.8 MB/s eta 0:00:00
   ━━

In [ ]:
from pycaret.classification import setup, compare_models

# Thiết lập môi trường PyCaret với tiền xử lý văn bản
clf_setup = setup(data=df, target='Emotion', session_id=123,
                  text_features=['Review'])  # Báo cho PyCaret biết cột nào chứa văn bản

# So sánh các mô hình
best_model = compare_models()


,Description,Value
0,Session id,123
1,Target,Emotion
2,Target type,Binary
3,Original data shape,"(16087, 2)"
4,Transformed data shape,"(16087, 7320)"
5,Transformed train set shape,"(11260, 7320)"
6,Transformed test set shape,"(4827, 7320)"
7,Text features,1
8,Preprocess,True
9,Imputation type,simple


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8920,0.9552,0.9045,0.8502,0.8764,0.7807,0.7821,7.5980
svm,SVM - Linear Kernel,0.8911,0.9536,0.9079,0.8462,0.8759,0.7791,0.7808,6.4900
lightgbm,Light Gradient Boosting Machine,0.8882,0.9539,0.8965,0.8481,0.8715,0.7727,0.7738,20.4060
xgboost,Extreme Gradient Boosting,0.8879,0.9534,0.9047,0.8424,0.8723,0.7727,0.7744,59.6410
et,Extra Trees Classifier,0.8869,0.9475,0.9003,0.8431,0.8707,0.7703,0.7718,43.9240
rf,Random Forest Classifier,0.8829,0.9470,0.8913,0.8415,0.8656,0.7619,0.7631,23.5640
ridge,Ridge Classifier,0.8812,0.9482,0.8798,0.8459,0.8623,0.7579,0.7585,24.5930
gbc,Gradient Boosting Classifier,0.8718,0.9432,0.8993,0.8167,0.8559,0.7410,0.7439,78.4570
ada,Ada Boost Classifier,0.8635,0.9367,0.8846,0.8105,0.8458,0.7238,0.7262,29.5750
dt,Decision Tree Classifier,0.8266,0.8272,0.7811,0.8036,0.7921,0.6434,0.6437,11.9950


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [ ]:
best_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:

from pycaret.classification import setup, compare_models, tune_model, evaluate_model, predict_model
# Tạo mô hình tốt nhất
tuned_model = tune_model(best_model)
# Đánh giá mô hình
evaluate_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8943,0.9567,0.9286,0.8387,0.8814,0.7865,0.7898
1,0.8899,0.9548,0.9097,0.8424,0.8747,0.7767,0.7786
2,0.8792,0.9497,0.9139,0.8208,0.8648,0.7562,0.7598
3,0.9023,0.9558,0.9412,0.8453,0.8907,0.8028,0.8066
4,0.8979,0.9593,0.9307,0.8438,0.8851,0.7936,0.7967
5,0.8872,0.9510,0.8952,0.8472,0.8705,0.7708,0.7717
6,0.8890,0.9604,0.9203,0.8346,0.8754,0.7757,0.7787
7,0.8766,0.9479,0.9203,0.8130,0.8633,0.7516,0.7564
8,0.8899,0.9549,0.9371,0.8262,0.8782,0.7784,0.7836


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
from pycaret.classification import *
save_model(tuned_model, 'tuned_model_new')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None, include=[],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('categorical_imputer',
                  TransformerWrapper(exclude=None, include=[],
                                     transformer=SimpleImputer(add_indicator...
                  TransformerWrapper(exclude=None, include=['Review'],
                                     transformer=EmbedTextFeatures(kwargs=None,
                             

In [ ]:
from pycaret.classification import *
save_model(best_model, 'best_model1')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None, include=[],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('categorical_imputer',
                  TransformerWrapper(exclude=None, include=[],
                                     transformer=SimpleImputer(add_indicator...
                  TransformerWrapper(exclude=None, include=['Review'],
                                     transformer=EmbedTextFeatures(kwargs=None,
                             

In [ ]:
from pycaret.classification import load_model, predict_model
import pandas as pd

# Chuẩn bị văn bản mới cần dự đoán
new_data = pd.DataFrame({
    'Review': ["Rất thích chế độ chơi game, không bị giật lag.	"]
})

# Dự đoán cảm xúc từ văn bản mới
predictions = predict_model(tuned_model, data=new_data)

# In kết quả dự đoán
print(predictions)


                                             Review  prediction_label  \
0  Rất thích chế độ chơi game, không bị giật lag.\t                 1   

   prediction_score  
0            0.6795  
